In [1]:
!pip install accelerate peft bitsandbytes transformers openpyxl accelerate peft bitsandbytes transformers trl trl -q

In [4]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os

In [5]:
model_id = "IlyaGusev/saiga_llama3_8b"

In [6]:
def get_model_and_tokenizer(model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
    model.config.use_cache = False
    model.config.pretraining_tp = 1
    return model, tokenizer

In [7]:
model, tokenizer = get_model_and_tokenizer(model_id)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/277 [00:00<?, ?B/s]

In [8]:
def formatted_prompt(input) -> str:
    prompt_template = [
        {
            "role": "system",
            "content": "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.",
        },
        {"role": "user", "content": input},
    ]
    return tokenizer.apply_chat_template(prompt_template, tokenize=False, add_generation_prompt=False)

In [9]:
from transformers import GenerationConfig
from time import perf_counter


def generate_response(user_input):
    prompt = formatted_prompt(user_input)
    inputs = tokenizer([prompt], return_tensors="pt")
    generation_config = GenerationConfig(
        penalty_alpha=0.6,
        do_sample=True,
        top_k=5,
        temperature=0.5,
        repetition_penalty=1.2,
        max_new_tokens=60,
        pad_token_id=tokenizer.eos_token_id,
    )
    start_time = perf_counter()
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, generation_config=generation_config)
    theresponse = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    output_time = perf_counter() - start_time
    print(f"Time taken for inference: {round(output_time,2)} seconds")

In [10]:
def formatted_train(input, response) -> str:
    prompt_template = [
        {
            "role": "system",
            "content": "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.",
        },
        {"role": "user", "content": input},
        {"role": "assistant", "content": response},
    ]
    return tokenizer.apply_chat_template(prompt_template, tokenize=False, add_generation_prompt=False)

In [11]:
import pandas as pd


df = pd.read_csv("QadAZz1L")
tasks = pd.read_excel("Fa16aI7H")

In [12]:
df = pd.merge(df, tasks, left_on="task_id", right_on="id", how="left").drop("id_y", axis=1)
df

,Unnamed: 0,id_x,task_id,student_solution,author_comment,author_comment_embedding,message,line_number,line,level,description,author_solution
0,0,13,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.2475823611021042 -1.3317935466766357 0.2535...,NaN,NaN,NaN,B,"Реализуйте программу, которая проверит, что цв...","logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4..."
1,1,14,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.25970256328582764 -1.4550446271896362 0.333...,NaN,NaN,NaN,B,"Реализуйте программу, которая проверит, что цв...","logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4..."
2,2,15,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.28293243050575256 -1.4774413108825684 0.243...,NaN,NaN,NaN,B,"Реализуйте программу, которая проверит, что цв...","logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4..."
3,3,16,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.44602400064468384 -0.845210611820221 -0.109...,NaN,NaN,NaN,B,"Реализуйте программу, которая проверит, что цв...","logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4..."
4,4,17,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Вы забыли поставить двоеточие после условия.,-0.2322002649307251 -1.3137400150299072 -0.383...,"Expected "":""",7.0,else,B,"Реализуйте программу, которая проверит, что цв...","logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4..."
...,...,...,...,...,...,...,...,...,...,...,...,...
342,342,741,47,"punctuation = '!#$%&()*+,-./:;<=>?@^_'\n\ndata...",Ошибка в открытых и скрытых тестах. \n\nВаш ко...,-0.2739786207675934 -1.7219487428665161 0.0072...,NaN,NaN,NaN,C,"Лолита изучает, какие студии дизайна чаще всег...","punctuation = '!#$%&()*+,-./:;<=>?@^_'\n\ndata..."
343,343,742,47,"punctuation = '!#$%&()*+,-./:;<=>?@^_'\n\ndata...",Ошибка в открытых и скрытых тестах. \n\nВаш ко...,-0.23899996280670166 -1.6277823448181152 -0.07...,NaN,NaN,NaN,C,"Лолита изучает, какие студии дизайна чаще всег...","punctuation = '!#$%&()*+,-./:;<=>?@^_'\n\ndata..."
344,344,743,47,"punctuation = '!#$%&()*+,-./:;<=>?@^_'\n\ndata...",Ошибка в открытых и скрытых тестах. \n\nВаш ко...,-0.23899996280670166 -1.6277823448181152 -0.07...,NaN,NaN,NaN,C,"Лолита изучает, какие студии дизайна чаще всег...","punctuation = '!#$%&()*+,-./:;<=>?@^_'\n\ndata..."
345,345,744,47,"punctuation = '!#$%&()*+,-./:;<=>?@^_'\n\ndata...",Ошибка в открытых и скрытых тестах. \n\nВаш ко...,-0.23899996280670166 -1.6277823448181152 -0.07...,NaN,NaN,NaN,C,"Лолита изучает, какие студии дизайна чаще всег...","punctuation = '!#$%&()*+,-./:;<=>?@^_'\n\ndata..."


In [13]:
def get_stuff(description, student_solution):
    return f"""Задание:
{description}

Попытка студента:
```python
{student_solution}
```

Объясни ошибку одним предложением.
"""

In [14]:
df["temp"] = df.apply(lambda x: get_stuff(x["description"], x["student_solution"]), axis=1)
df["temp"]

0      Задание:\nРеализуйте программу, которая провер...
1      Задание:\nРеализуйте программу, которая провер...
2      Задание:\nРеализуйте программу, которая провер...
3      Задание:\nРеализуйте программу, которая провер...
4      Задание:\nРеализуйте программу, которая провер...
                             ...                        
342    Задание:\nЛолита изучает, какие студии дизайна...
343    Задание:\nЛолита изучает, какие студии дизайна...
344    Задание:\nЛолита изучает, какие студии дизайна...
345    Задание:\nЛолита изучает, какие студии дизайна...
346    Задание:\nЛолита изучает, какие студии дизайна...
Name: temp, Length: 347, dtype: object

In [15]:
df["text"] = df.apply(lambda x: formatted_train(x["temp"], x["author_comment"]), axis=1)

In [16]:
data = Dataset.from_pandas(df)

In [17]:
print(df["text"].loc[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.<|eot_id|><|start_header_id|>user<|end_header_id|>

Задание:
Реализуйте программу, которая проверит, что цвет используется только в проекте по созданию логотипа, но не в проекте по созданию дизайна сайта:

Даны два списка logo_project и cite_project с кодами используемых цветов (строки).
В переменную color считывается код цвета (строка). Этот код уже написан.
Программа должна проверять, что код цвета color есть только в списке logo_project, и если да, то печатать True. 
В остальных случаях программа печатает False. 

Попытка студента:
```python
logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4', '#e4b3cd', '#e4e3b3', '#c0ced7']
cite_project = ['#e4e3b3', '#a7a8f0', '#ccb1e6', '#b4f99e', '#f9b59e', '#c0ced7']

color = input()

if color in logo_project and color in cite_project:
    print(True)
else:
    print(False)
```

Объясни ошибку о

In [18]:
peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


training_arguments = TrainingArguments(
    output_dir="my_model",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=10,
    num_train_epochs=3,
    max_steps=250,
    fp16=True,
    push_to_hub=False,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    dataset_text_field="text",
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=1024,
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/347 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,1.799500
20,1.316400
30,0.962800
40,0.680400
50,0.482100
60,0.339800
70,0.248300
80,0.194600
90,0.162700
100,0.145500


In [20]:
def generate_response(user_input):
    prompt = formatted_prompt(user_input)
    inputs = tokenizer([prompt], return_tensors="pt")
    generation_config = GenerationConfig(
        penalty_alpha=0.6,
        do_sample=True,
        top_k=5,
        temperature=0.5,
        repetition_penalty=1.2,
        max_new_tokens=60,
        pad_token_id=tokenizer.eos_token_id,
    )
    start_time = perf_counter()
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, generation_config=generation_config)
    theresponse = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    output_time = perf_counter() - start_time
    print(f"Time taken for inference: {round(output_time,2)} seconds")

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4

In [25]:
print(df["text"].loc[15])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.<|eot_id|><|start_header_id|>user<|end_header_id|>

Задание:
Реализуйте программу, которая определит количество корректных названий проектов для конкурса дизайнеров интерьера:

В цикле while считываются названия проектов до тех пор, пока не введена строка "СТОП".
Гарантируется, что названия проектов не повторяются. 
Гарантируется, что до строки "СТОП" будет введена как минимум одна строка.
Напечатайте количество проектов, которые состоят только из заглавных букв и в которых нет знака подчеркивания.

Попытка студента:
```python
result = 0

while True:
    info = input()
    if info == 'СТОП':
    break
    
    if '_' not in info and info.isupper():
        result += 1
        
print(result)
```

Объясни ошибку одним предложением.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Обратите внимание на ошибку в табуляции выражения

In [26]:
generate_response(df["text"].loc[15])

system

Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.user

system

Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.user

Задание:
Реализуйте программу, которая определит количество корректных названий проектов для конкурса дизайнеров интерьера:

В цикле while считываются названия проектов до тех пор, пока не введена строка "СТОП".
Гарантируется, что названия проектов не повторяются. 
Гарантируется, что до строки "СТОП" будет введена как минимум одна строка.
Напечатайте количество проектов, которые состоят только из заглавных букв и в которых нет знака подчеркивания.

Попытка студента:
```python
result = 0

while True:
    info = input()
    if info == 'СТОП':
    break
    
    if '_' not in info and info.isupper():
        result += 1
        
print(result)
```

Объясни ошибку одним предложением.assistant

Обратите внимание на ошибку в табуляции выражения break.assistant

Скорректированная п

In [30]:
from huggingface_hub import login

login()

In [31]:
model.push_to_hub("llama-test", token="")
tokenizer.push_to_hub("llama-test", token="")

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/seara/llama-test/commit/69f8d87a5096536a9e7cfc548a943ddb2b513c3e', commit_message='Upload tokenizer', commit_description='', oid='69f8d87a5096536a9e7cfc548a943ddb2b513c3e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/seara/llama-test', endpoint='https://huggingface.co', repo_type='model', repo_id='seara/llama-test'), pr_revision=None, pr_num=None)

In [43]:
df = pd.read_csv("3WyLKvgF")
tasks = pd.read_excel("g2ombpyq")
df = pd.merge(df, tasks, left_on="task_id", right_on="id", how="left").drop("id_y", axis=1)


def get_stuff(description, student_solution):
    return f"""Задание:
{description}

Попытка студента:
```python
{student_solution}
```

Объясни ошибку одним предложением.
"""


df["temp"] = df.apply(lambda x: get_stuff(x["description"], x["student_solution"]), axis=1)

In [46]:
!pip install tqdm -q

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [48]:
# Исключительно ознакомительный пример.
# НЕ НАДО ТАК ИНФЕРИТЬ МОДЕЛЬ В ПРОДЕ.
# См. https://github.com/vllm-project/vllm или https://github.com/huggingface/text-generation-inference

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from tqdm.notebook import tqdm

MODEL_NAME = "IlyaGusev/saiga_llama3_8b"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."


model.eval()

results = []

inputs = df["temp"].tolist()
for query in tqdm(inputs):
    generation_config = GenerationConfig(
        penalty_alpha=0.6,
        do_sample=True,
        top_k=5,
        temperature=0.1,
        repetition_penalty=1.2,
        max_new_tokens=60,
        pad_token_id=tokenizer.eos_token_id,
    )
    prompt = tokenizer.apply_chat_template(
        [{"role": "system", "content": DEFAULT_SYSTEM_PROMPT}, {"role": "user", "content": query}],
        tokenize=False,
        add_generation_prompt=True,
    )
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]) :]
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    results.append(output)
    print(output)
    print()
    print("==============================")
    print()

  0%|          | 0/325 [00:00<?, ?it/s]

Ошибка при форматировании данных. Проверьте отображение значений переменных в кавычках стринг. Попробуйте изменить print-statement, чтобы исправить ошибку.


Ошибка при форматировании данных внутри функции print(). Вы забыли поставить двоеточие после первой конструкции фразы. Попробуйте исправить этот дефект.


Вы забыли поставить двоеточие после функции print(). Попробуйте изменить код.


Ошибка при форматировании данных. Проверьте табуляцию параметра функции print(). Помните, что условие if не выполняется. Попробуйте изменить параметр print(), чтобы ошибку исправили.


Ошибка при форматировании данных. Проверьте табуляцию переменных перед выводом информации. Например, убедитесь что переменная money выполянет функцию format().


Вы забыли поставить двойные кавычки вокруг значения, которое необходимо вернуть функции print(). Попробуйте изменить код таким образом, чтобы корректно выполнилась функция print().


Ошибка при форматировании данных. Не забывайте применять f-строки к каждому п

In [49]:
print("END")

END


In [50]:
df["results"] = results

In [51]:
df.to_csv("HAHA.csv")